In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime, timedelta
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
from openpyxl import *

# ChromeDriver 경로 설정 (macOS 사용자의 경우 절대 경로로 설정 필요)
chromedriver_path = '/opt/homebrew/bin/chromedriver'  # 실제 chromedriver 경로로 변경

# Selenium 4.x에서 Service 객체 사용
service = Service(chromedriver_path)

# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--disable-dev-shm-usage")  # 메모리 제한 해제
chrome_options.add_argument("--no-sandbox")            # 샌드박스 모드 비활성화
chrome_options.add_argument("--headless")              # 브라우저를 헤드리스 모드로 실행 (필요에 따라 삭제)

# 네이버 뉴스 페이지
link = 'https://news.naver.com/breakingnews/section/101/259?date='

# 시작 날짜와 종료 날짜 정의
start_date = datetime(2023, 12, 1)
end_date = datetime(2023, 12, 31)  # 테스트 범위를 줄여 설정 (원래: 2024, 11, 30)

# 날짜 리스트 생성
date_list = [(start_date + timedelta(days=x)).strftime('%Y%m%d') for x in range((end_date - start_date).days + 1)]

# 날짜별 크롤링
for date in tqdm(date_list):
    retries = 3  # 재시도 횟수
    while retries > 0:
        try:
            # 새로운 드라이버 생성
            driver = webdriver.Chrome(service=service, options=chrome_options)

            # 날짜별 링크 이동
            main_link = link + date
            driver.get(main_link)
            time.sleep(3)

            # 더보기 버튼 클릭
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'section_more_inner._CONTENT_LIST_LOAD_MORE_BUTTON'))
                )
                while True:
                    try:
                        more_button = driver.find_element(By.CLASS_NAME, 'section_more_inner._CONTENT_LIST_LOAD_MORE_BUTTON')
                        more_button.click()
                        time.sleep(3)
                    except Exception:
                        break
            except Exception as e:
                print("More button not found or no more articles to load:", e)

            # 기사 추출
            articles = driver.find_elements(By.CLASS_NAME, 'sa_text_title')
            data = [[i + 1, article.text.strip(), article.get_attribute('href')] for i, article in enumerate(articles)]

            # DataFrame 생성 및 저장
            Main_link = pd.DataFrame(data, columns=['number', 'title', 'link'])
            excel_name = f'./crawl/news_{date}.xlsx'
            Main_link.to_excel(excel_name, index=False)
            print(f'엑셀 파일 저장 완료: {excel_name}')

            break  # 작업 성공 시 루프 종료
        except Exception as e:
            print(f"Retrying date {date}, attempts left: {retries-1}, Error: {e}")
            retries -= 1
            time.sleep(5)  # 재시도 전 대기
        finally:
            driver.quit()

    if retries == 0:
        print(f"Failed to process date {date} after multiple retries.")


  0%|          | 0/31 [00:00<?, ?it/s]

엑셀 파일 저장 완료: ./crawl/news_20231201.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231202.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231203.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231204.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231205.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231206.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231207.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231208.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231209.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231210.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231211.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231212.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231213.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231214.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231215.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231216.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231217.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231218.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231219.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231220.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231221.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231222.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231223.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231224.xlsx
엑셀 파일 저장 완료: ./crawl/news_20231225.xlsx


In [ ]:
# 시작 날짜와 종료 날짜 정의
start_date = datetime(2023, 12, 1)
end_date = datetime(2023, 12, 31)

# 날짜 리스트 생성
date_list = [(start_date + timedelta(days=x)).strftime('%Y%m%d') for x in range((end_date - start_date).days + 1)]

for date in tqdm(date_list):
    # 첫 번째 코드에서 지정한 뉴스의 링크들이 담긴 파일
    link = pd.read_excel(f'./crawl/news_{date}.xlsx')
    # 엑셀 파일의 빈칸이 있기에 최종 결과파일을 별도로 생성
    excel_name = f'./newsdata/news_detail_{date}.xlsx'
    Main_link = list(link['link'])

    # number: 기사 순서, title: 기사 제목, information: 본문 내용, link: 기사의 링크
    Information = pd.DataFrame({'number': [], 'title': [], 'information': [], 'link': []})
    # 본문 내용을 추가하기 전이기 때문에 미리 나머지 내용을 담아둠
    Information['number'] = link['number']
    Information['title'] = link['title']
    Information['link'] = link['link']
    information = []

    for main_link in Main_link:
        # 기사가 전체적으로 2개의 구조를 가지고 있음 (게임/리뷰 카테고리에 한하여)
        # 하나의 구조를 기준으로 삼고, 해당 부분에서 오류가 발생하면 다음 구조의 기사로 판단
        try:
            response = requests.get(main_link, headers={"User-Agent": "Mozilla/5.0"})
            if response.status_code == 200:
                html = response.content
                soup = BeautifulSoup(html, 'html.parser')
                # 기사의 본문 내용만 담고 있는 부분
                info = soup.find('div', {'id': 'newsct_article'}).text.strip()
                # 기사 내용 데이터 분석을 위해 줄바꿈을 띄어쓰기로 변경
                info = info.replace('\n', ' ')
                information.append(info)
        except:
            # 다른 구조에서 재 크롤링 코드
            # 여기서 오류가 나는 경우는 게임/리뷰 기사가 아닌 다른 카테고리의 기사로 판단
            try:
                response = requests.get(main_link, headers={"User-Agent": "Mozilla/5.0"})
                if response.status_code == 200:
                    html = response.content
                    soup = BeautifulSoup(html, 'html.parser')
                    # 기사의 본문 내용을 담고 있는 부분
                    info = soup.find('div', {'id': 'newsEndContents'}).text.strip()
                    info = info.replace('\n', ' ')
                    # 해당 구조의 기사는 기자의 정보가 본문과 무조건 같이 존재
                    # 기자의 정보 부분은 필요가 없기 때문에 기자 정보의 기준점이 되는 부분을 찾음
                    # 기자의 정보 기준이 기재정보라는 단어이기 때문에 그 이후는 삭제
                    '''
                    end = info.index('기재정보')
                    info = info[:end]
                    '''
                    information.append(info)
            except Exception as e:
                info = ''
                information.append(info)
                # 오류가 발생하는 이유와 발생하는 링크를 출력하여 오류를 확인하는 장치
                # print(e)
                # print(main_link)

    Information['information'] = information
 
    with pd.ExcelWriter(excel_name) as writer:
        Information.to_excel(writer, sheet_name='결과값', index=False)
        print(f'엑셀 파일 저장 완료: {excel_name}')